# Movie Recommendation System

#### Load the CSV files into dataframe

In [ ]:
import os

In [ ]:
data_dir: str = "data"

In [ ]:
ratings_csv: str = os.path.join(data_dir, "ratings.csv")
movies_csv: str = os.path.join(data_dir, "movies.csv")

In [ ]:
import pandas as pd
import numpy as np
from typing import Any

In [ ]:
movies_dtype: dict[str, Any] = {
    "movieId": np.int32,
    "title": str,
    "genres": str
}

In [ ]:
movies_df: pd.DataFrame = pd.read_csv(movies_csv, dtype=movies_dtype)

In [ ]:
DF_CHUNK_SIZE: int = 100_000

In [ ]:
from pandas.io.parsers.readers import TextFileReader

In [ ]:
ratings_dtype: dict[str, Any] = {
    "userId": np.int32,
    "movieId": np.int32,
    "rating": np.float16,
    "timestamp": np.int32
}

In [ ]:
ratings_df_chunks: TextFileReader = pd.read_csv(ratings_csv, 
                                                chunksize=DF_CHUNK_SIZE, 
                                                dtype=ratings_dtype)

In [ ]:
movies_df.head()

#### One hot encode the genres

In [ ]:
movies_df["genres"] = movies_df["genres"].str.split("|")

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
mlb: MultiLabelBinarizer = MultiLabelBinarizer()
mlb.fit_transform(movies_df["genres"])

In [ ]:
cache_dir: str = "cache"

In [ ]:
mlb_classes_csv = os.path.join(cache_dir, "mlb_classes.csv")

In [ ]:
mlb_classes_df = pd.DataFrame({"pos": i, "class": c} 
                              for i, c in enumerate(mlb.classes_))

In [ ]:
mlb_classes_df.to_csv(mlb_classes_csv)

In [ ]:
with open(mlb_classes_csv, "r") as f:
    mlb = MultiLabelBinarizer(classes=pd.read_csv(mlb_classes_csv)["class"])

In [ ]:
encodings = mlb.fit_transform(movies_df["genres"])
genre_bits = np.zeros(len(movies_df), dtype=np.int32)

for i in range(encodings.shape[1]):
    genre_bits = genre_bits | (encodings[:, i].astype(np.int32) << i)

In [ ]:
movies_df["genres"] = genre_bits

In [ ]:
movies_df.head()

#### Collect user input

In [ ]:
input_df = pd.read_csv(os.path.join(data_dir, "input.csv"))

In [ ]:
input_df.head()